In [ ]:
__doc__ = """
This notebook appears to have been replaced by L5,L3 by aL5,aL4 estimation.ipynb,
and is no longer useful.
"""

In [ ]:
import os, re, matplotlib, pandas, collections, importlib, sys
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats as scs

import __init__
sys.path = __init__.add_paths(sys.path)

from importlib import reload
import sameRiver
from sameRiver import *

import sameRiver.parameters_for_hyperbolic_curve as pfhc
import importlib
import utils
importlib.reload(pfhc)
importlib.reload(utils)

In [ ]:
def purification_control_oligos(df):
    
    print("\n----\nPCO:")
    # All purification oligos, which are all on the 800 wavelength.
    pco = df[df['Object']=='PCO']
    pco = pco[pco['Channel']==800]

    # Fit a line to the standards, if possible.
    pco_stds = pco[pco['STD?']=='STD']
    print('PCO: {0} PCO standards at {1} fmols'.format(
        len(pco_stds), set(pco_stds['fmols'])))
    
    # Line fit.
    signals = pco_stds['Signal'].tolist() + [0]
    fmols = pco_stds['fmols'].tolist() + [0]
    a = scs.linregress(signals, fmols)
    m = a.slope
    b = a.intercept
    print('fmols=m*signal+b for PCO: m {0} b {1}'.format(m, b))
    
    # Estimate the purification efficiency.
    pco_exp = pco[pco['STD?']!='STD']
    pco_exp['Est. fmols'] = [m*x + b for x in pco_exp['Signal']]
    pco_exp['Purification efficiency (%)'] = [100*(x/y) for x, y in zip(
        pco_exp['Est. fmols'], pco_exp['Input PCO fmols'])]
    
    print('--- pco_exp:')
    print(pco_exp)
    df['Purification efficiency (%)'] = 0
    for n in pco_exp.index:
        df.loc[n, 'Purification efficiency (%)'] = pco_exp.loc[n, 'Purification efficiency (%)']
    print('\n\n\n41:')
    try:
        print(df.loc[41])
    except:
        print('not in index')
        
    return df, pco_stds, pco_exp


def total_L3_on_gel(df):
    df, pco_stds, pco_exp = purification_control_oligos(df)

    print("----\n" * 5)
    print('total_L3_on_gel():')
    purification_to_eff = dict(zip(
        pco_exp['Purification'], pco_exp['Purification efficiency (%)']))

    print("purification_to_eff:")
    print(purification_to_eff)
#    aLL = df.loc[[(x in ['αL5', 'αL3']) for x in df.Object]]
    aLL = df[df['Object']=='αL3']
    aLL = aLL[aLL['Complex']!='Staple']
    aLL = aLL[aLL['STD?']!='STD']
    

    if 'L5' not in purification_to_eff:
        purification_to_eff['L5'] = 1.
    aLL['Total L3 input to qRNA purification'] = [
        fmols/(purification_to_eff[purified]/100) for fmols, purified in zip(
            aLL['Est. fmols'], aLL['Purification'])]
    
    print('est fmols, purification:')
    print([(x,y) for x,y in zip(
            aLL['Est. fmols'], aLL['Purification'])])
    
    print("aLL['Total L3 input to qRNA purification'] : ")
    print(aLL['Total L3 input to qRNA purification'])
    
    aLL['Total L3 on gel']  = [
        L3_input_to_purification/(fraction_PK_input_to_purification
        ) for L3_input_to_purification, fraction_PK_input_to_purification in zip(
        aLL['Total L3 input to qRNA purification'], aLL['Input % PK extract'])
    ]
    print('total L3 input to qRNA purification, input % PK extract:')
    print([(x,y) for x,y in zip(
        aLL['Total L3 input to qRNA purification'], aLL['Input % PK extract'])])
    
    print('Total L3 on gel:')
    print(aLL['Total L3 on gel'])
    
    print("end total_L3_on_gel----\n")
    return aLL

def ligation_efficiency(df):
    
    by_rep = collections.defaultdict(dict)
    
    for row in df.to_dict('records'):
        by_rep[row['Protein CLIP replicate']][(row['Purification'], row['Object'])
                                ] = row['Est. fmols']
        
    _d = collections.defaultdict(dict)
    results = []
    
    for rep in by_rep:

        #if ('L3', 'αL5') not in by_rep[rep]:
        #    continue

        if ('L3', 'αL5') in by_rep[rep]:
            
            _d[rep, "5' ligation efficiency"
                   ] = by_rep[rep][('L3', 'αL5')]/by_rep[rep][('L3', 'αL3')]
            
            results.append({
                'Replicate': rep,
                'Ligation': "5'",
                'Efficiency': 100 * by_rep[rep][('L3', 'αL5')]/by_rep[rep][('L3', 'αL3')],
            })           
            
        if ('L5', 'αL3') in by_rep[rep]:
            
            _d[rep, "3' ligation efficiency"
                   ] = by_rep[rep][('L5', 'αL3')]/by_rep[rep][('L5', 'αL5')]
        
            results.append({
                'Replicate': rep,
                'Ligation': "3'",
                'Efficiency': 100 * by_rep[rep][('L5', 'αL3')]/by_rep[rep][('L5', 'αL5')]
            })

    eff = pandas.DataFrame(results)

    return eff

In [ ]:
importlib.reload(pfhc)
importlib.reload(utils)
def ests_Kas(
    df, params_file='/Users/dfporter/pma/nonclip/180523.xlsx', params_sheet='180523_1'):
    
    staples = df[df['Complex']=='Staple'].copy()

    params5, params3 = pfhc.get_parameters(fname=params_file, sheet_name=params_sheet)
    df['Replicate'] = ['R' + str(x)[:1] for x in df.Replicate]

    df, K = utils.get_Ka(df)
    
    l5 = df[df['Object']=='αL5'].copy()
    l3 = df[df['Object']=='αL3'].copy()
    
    non_l5l3 = df[df['Object']!='αL5'].copy()
    non_l5l3 = non_l5l3[non_l5l3['Object']!='αL3'].copy()

    l5 = pfhc.est_fmols(l5, params5, channel=800, staples=staples)
    l3 = pfhc.est_fmols(l3, params3, channel=700, staples=staples)

    df = pandas.concat([non_l5l3, l5, l3])
    
    #print(l5)
    #print(l3)
    
    return df

obj_color = utils.obj_color

aL3 = utils.load_sheet(
    fname='../clip/experiments/exp37/exp37.xlsx',
#    sheet_name='180621_qRNA'
    sheet_name='180622_2_qRNA'
)

aL3 = ests_Kas(aL3)
aL3, pco_stds, pco_exp = purification_control_oligos(aL3)
aLL = total_L3_on_gel(aL3)

aL5 = utils.load_sheet(
    fname='../clip/experiments/exp37/exp37.xlsx',
    sheet_name='180622_1_qRNA')

aL5 = ests_Kas(aL5)
aL5.to_excel('../test.xlsx')


In [ ]:
df = aL5 #pandas.concat([aL3, aL5])
#print(df)
eff = ligation_efficiency(df)
print(eff)
plt.clf()
fig, ax = plt.subplots()
sns.stripplot(
    data=eff, x='Efficiency', y='Ligation', #hue='Ligation',
)#palette=obj_color)
fig.savefig('../figs/hnRNPC_lig_eff.pdf')
plt.show()
plt.clf()

df2 = df[df['Complex']!='Staple']
fig, ax = plt.subplots()
sns.stripplot(
    data=df2[df['STD?']!='STD'], x='Est. fmols', y='Object', hue='Purification',
    dodge=True,
    palette=obj_color)
fig.savefig('../figs/hnRNPC_quant_RNA.pdf')
plt.show()
plt.clf()
